In [100]:
import pandas as pd

df = pd.read_csv(r"C:\Users\Yuval\Desktop\Hotels_data_Changed.csv")
df.columns
df.head(5)

,Snapshot ID,Snapshot Date,Checkin Date,Days,Original Price,Discount Price,Discount Code,Available Rooms,Hotel Name,Hotel Stars,WeekDay,DiscountDiff,DiscountPerc,DayDiff
0,1,2015-07-17,2015-08-12,5,1178,1040,1,6,Best Western Plus Seaport Inn Downtown,3,Wed,138,11.714771,26
1,1,2015-07-17,2015-08-19,5,1113,982,1,8,Best Western Plus Seaport Inn Downtown,3,Wed,131,11.769991,33
2,1,2015-07-17,2015-08-13,5,4370,4240,1,3,The Peninsula New York,5,Thu,130,2.974828,27
3,1,2015-07-17,2015-07-26,5,1739,1667,1,18,Eventi Hotel a Kimpton Hotel,4,Sun,72,4.140311,9
4,1,2015-07-17,2015-08-12,5,1739,1672,1,3,Eventi Hotel a Kimpton Hotel,4,Wed,67,3.852789,26


In [140]:
df.sort_values(['DiscountPerc'], ascending=True, inplace=True)
df.drop_duplicates(subset=['Hotel Name', 'DayDiff', 'Checkin Date', 'Snapshot Date', 'WeekDay', 'Discount Code'], keep='first', inplace=True)
df.to_csv(r"C:\Users\Yuval\Desktop\Hotels_data_filtered.csv")
df.head(5)

,Snapshot ID,Snapshot Date,Checkin Date,Days,Original Price,Discount Price,Discount Code,Available Rooms,Hotel Name,Hotel Stars,WeekDay,DiscountDiff,DiscountPerc,DayDiff
12240,14,2015-07-22,2015-07-23,5,17200,17100,2,3,The Peninsula New York,5,Thu,100,0.581395,1
14998,15,2015-07-23,2015-07-27,5,17200,17100,3,3,The Peninsula New York,5,Mon,100,0.581395,4
82897,446,2015-10-10,2015-10-23,5,5750,5695,1,10,The Plaza Hotel,5,Fri,55,0.956522,13
79857,441,2015-10-07,2015-10-10,5,5625,5570,4,10,The Plaza Hotel,5,Sat,55,0.977778,3
53771,361,2015-09-17,2015-10-19,5,5625,5570,4,10,The Plaza Hotel,5,Mon,55,0.977778,32


In [141]:
# Naive Bayes
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

vectorizer = CountVectorizer()
data = map(lambda x: ','.join(str(i) for i in x), df[['Hotel Name', 'DayDiff', 'Checkin Date', 'Snapshot Date', 'WeekDay']].values)
counts = vectorizer.fit_transform(data)
classifier = MultinomialNB()
targets = df['Discount Code'].values
classifier.fit(counts, targets)

#print(counts)



MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [153]:
# Decision tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import LabelEncoder


data_for_tree = df.copy()
del data_for_tree['Snapshot ID']
del data_for_tree['Days']
del data_for_tree['Original Price']
del data_for_tree['Discount Price']
del data_for_tree['Available Rooms']
del data_for_tree['Hotel Stars']
del data_for_tree['DiscountDiff']

le = LabelEncoder()

le_weekday = LabelEncoder()
le_weekday.fit(data_for_tree['WeekDay'].unique())
data_for_tree['WeekDay'] = le_weekday.transform(data_for_tree['WeekDay'].values)

le_hotel = LabelEncoder()
le_hotel.fit(data_for_tree['Hotel Name'].unique())
data_for_tree['Hotel Name'] = le_hotel.transform(data_for_tree['Hotel Name'].values)

le_ci_date = LabelEncoder()
le_ci_date.fit(data_for_tree['Checkin Date'].unique())
data_for_tree['Checkin Date'] = le_ci_date.transform(data_for_tree['Checkin Date'].values)

le_ss_date = LabelEncoder()
le_ss_date.fit(data_for_tree['Snapshot Date'].unique())
data_for_tree['Snapshot Date'] = le_ss_date.transform(data_for_tree['Snapshot Date'].values)

X = data_for_tree.values
Y = data_for_tree.values

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3, random_state = 100)


dt = DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=5,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=100, splitter='best')
dt.fit(X_train,y_train)


TypeError: __init__() got an unexpected keyword argument 'class_weight'